# Ensemble methods. Stacking


Stacking method use several classifiers and use them to generate an input feature matrix for a stacked classifier that do the final prediction.

Let's load the data set first.

In [3]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

To simplify the notebook, we use methods that are available in scikit-learn package. We load only several, but in the exercise you gonna need to load other methods as well.

## Stacking

We have the following steps:    

* create $T$ classifiers and learn each to get $m$ predictions (hypothesis $h_{t}$,
* construct data set of predictions and construct a new classifier $C_{m}$ for each dataset,
* construct a $C_{h}$ classifier that combines all $C_{m}$ classifiers.

First of all, let's import required libraries:

In [4]:
import numpy as np
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In the first part we build three different models based on three different classifiers:

In [5]:
def build_classifiers():
    
    neighbors = KNeighborsClassifier()
    neighbors.fit(data_set, labels)

    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)

    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)

    return neighbors, linear_regression, qda

Based on the classifiers prediction, we build a feature vector for the decision tree classifier. Finally, we train and predict with the stacked classifier.

In [6]:
def build_stacked_classifier(classifiers):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    decision_tree = DecisionTreeClassifier()
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    decision_tree.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = decision_tree.predict(test_set)
    return predicted

Stacked classifier accuracy can be measured as below:

In [7]:
classifiers = build_classifiers()
predicted = build_stacked_classifier(classifiers)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.7


In this case, the three used classifiers does not give any value, because we get a higher value using just the decision treee classifier.

In [8]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(data_set, labels)
predicted = decision_tree.predict(test_data_set)
accuracy = accuracy_score(test_labels, predicted)
print(accuracy)

0.9


## Grading

Grading is a stacking type where we train a grading classifier that checks if our classifier is good or not on the prediction. We need to build classifiers like we did in the previous example:

In [ ]:
def build_classifiers():

    neighbors = KNeighborsClassifier()
    neighbors.fit(data_set, labels)

    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)

    return neighbors, qda

In the method below we get a vector of model grads:

In [ ]:
def calculate_accuracy_vector(predicted, labels):
    result = []
    for i in range(len(predicted)):
        if predicted[i] == labels[i]:
            result.append(1)
        else:
            result.append(0)
    return result

Finally, we can build our grading meta classifier:

In [ ]:
def build_grading_classifier(classifiers):
    output = []
    matrix = []
    for classifier in classifiers:
        predicted = classifier.predict(data_set)
        output.append(predicted)
        matrix.append(calculate_accuracy_vector(predicted,labels))

    grading_classifiers = []
    for i in range(len(classifiers)):
        tree = DecisionTreeClassifier()
        tree.fit(data_set, matrix[i])
        grading_classifiers.append(tree)
    return grading_classifiers

Test prediction function takes the model and grading model and return the prediction and grad of the prediction:

In [ ]:
def test_prediction(classifiers, grading_classifiers, i):
    prediction = classifiers[i].predict(test_data_set)
    grad = grading_classifiers[i].predict(test_data_set)
    return prediction, grad

We can test now the output, labels and grads of the output:

In [ ]:
classifiers = build_classifiers()
grading_classifiers = build_grading_classifier(classifiers)
prediction, grad = test_prediction(classifiers, grading_classifiers, 0)

print(prediction)
print(grad)
print(test_labels)